# 生成电视剧剧本

在这个项目中，你将使用 RNN 创作你自己的[《辛普森一家》](https://zh.wikipedia.org/wiki/%E8%BE%9B%E6%99%AE%E6%A3%AE%E4%B8%80%E5%AE%B6)电视剧剧本。你将会用到《辛普森一家》第 27 季中部分剧本的[数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)。你创建的神经网络将为一个在 [Moe 酒馆](https://simpsonswiki.com/wiki/Moe's_Tavern)中的场景生成一集新的剧本。

## 获取数据
我们早已为你提供了数据`./data/Seinfeld_Scripts.txt`。我们建议你打开文档来看看这个文档内容。

>* 第一步，我们来读入文档，并看几段例子。
* 然后，你需要定义并训练一个 RNN 网络来生成新的剧本！

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## 探索数据
使用 `view_line_range` 来查阅数据的不同部分，这个部分会让你对整体数据有个基础的了解。你会发现，文档中全是小写字母，并且所有的对话都是使用 `\n` 来分割的。

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

In [3]:
# check dafa source

lines[:10]

['jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! you wanna go to sleep, you wanna get up, you wanna go out again tomorrow, right? where ever you are in life, its my feeling, youve gotta go. ',
 '',
 'jerr

---
## 实现预处理函数
对数据集进行的第一个操作是预处理。请实现下面两个预处理函数：

- 查询表
- 标记符号

### 查询表
要创建词嵌入，你首先要将词语转换为 id。请在这个函数中创建两个字典：

- 将词语转换为 id 的字典，我们称它为 `vocab_to_int`
- 将 id 转换为词语的字典，我们称它为 `int_to_vocab`

请在下面的元组中返回这些字典
 `(vocab_to_int, int_to_vocab)`

In [4]:
import problem_unittests as tests
from collections import Counter
import itertools

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    text = Counter(text)
    word2int = {v[0]:k for k,v in enumerate(sorted(text.items(), key = lambda item: item[1], reverse=True))}
    int2word = {v:k for k,v in word2int.items()}
    
    # return tuple
    return (word2int, int2word)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### 标记符号的字符串
我们会使用空格当作分隔符，来将剧本分割为词语数组。然而，句号和感叹号等符号使得神经网络难以分辨“再见”和“再见！”之间的区别。

实现函数 `token_lookup` 来返回一个字典，这个字典用于将 “!” 等符号标记为 “||Exclamation_Mark||” 形式。为下列符号创建一个字典，其中符号为标志，值为标记。

- period ( . )
- comma ( , )
- quotation mark ( " )
- semicolon ( ; )
- exclamation mark ( ! )
- question mark ( ? )
- left parenthesis ( ( )
- right parenthesis ( ) )
- dash ( -- )
- return ( \n )

这个字典将用于标记符号并在其周围添加分隔符（空格）。这能将符号视作单独词汇分割开来，并使神经网络更轻松地预测下一个词汇。请确保你并没有使用容易与词汇混淆的标记。与其使用 “dash” 这样的标记，试试使用“||dash||”。

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token = {'.':'period',
            ',':'comma',
            '"':'quotation_mark',
            '!':'exclamation_mark',
            '?':'question_mark',
            '(':'left_parenthesis',
            ')':'right_parenthesis',
            '-':'dash',
            '\n':'return',
            ';':'semicolon'}
    
    markdown = lambda x: '||{}||'.format(x.upper())
        
    return {k:markdown(v) for k,v in token.items()}

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [6]:
token_lookup()

{'.': '||PERIOD||',
 ',': '||COMMA||',
 '"': '||QUOTATION_MARK||',
 '!': '||EXCLAMATION_MARK||',
 '?': '||QUESTION_MARK||',
 '(': '||LEFT_PARENTHESIS||',
 ')': '||RIGHT_PARENTHESIS||',
 '-': '||DASH||',
 '\n': '||RETURN||',
 ';': '||SEMICOLON||'}

## 预处理并保存所有数据
运行以下代码将预处理所有数据，并将它们保存至文件。建议你查看`helpers.py` 文件中的 `preprocess_and_save_data` 代码来看这一步在做什么，但是你不需要修改`helpers.py`中的函数。

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# 检查点
这是你遇到的第一个检点。如果你想要回到这个 notebook，或需要重新打开 notebook，你都可以从这里开始。预处理的数据都已经保存完毕。

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## 创建神经网络
在本节中，你会构建 RNN 中的必要 Module，以及 前向、后向函数。

### 检查 GPU 访问权限

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## 输入
让我们开始预处理输入数据。我们会使用 [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) 来为数据库提供一个数据格式；以及一个 [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), 该对象会实现 batching，shuffling 以及其他数据迭代功能。

你可以通过传入 特征 和目标 tensors 来创建 TensorDataset，随后创建一个 DataLoader 。
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
 通过 `TensorDataset` 和 `DataLoader` 类来实现  `batch_data` 函数来将 `words` 数据分成 `batch_size` 批次。

>你可以使用 DataLoader 来分批 单词, 但是你可以自由设置 `feature_tensors` 和 `target_tensors` 的大小以及 `sequence_length`。

比如，我们有如下输入:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

你的第一个 `feature_tensor` 会包含:
```
[1, 2, 3, 4]
```
随后的 `target_tensor` 会是接下去的一个字符值:
```
5
```
那么，第二组的`feature_tensor`, `target_tensor` 则如下所示:
```
[2, 3, 4, 5]  # features
6             # target
```

In [10]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    feature = []
    target = []
    for idx, w in enumerate(words):
        try:
            t = words[idx + sequence_length]
            f = words[idx:idx+sequence_length]
            feature.append(f)
            target.append(t)
        except IndexError as e:
            break
    
    data = TensorDataset(torch.from_numpy(np.asarray(feature)).long(), torch.from_numpy(np.asarray(target)).long())
    dataloader = DataLoader(data, batch_size, shuffle = True)
    
    # return a dataloader
    return dataloader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


In [11]:
next(iter(batch_data(int_text, 5, 2)))

[tensor([[ 202,   25,    1,    5,   42],
         [  18,    3,    0,    0,   16]]), tensor([ 1062,    32])]

In [12]:
print(len(vocab_to_int))

21388


### 测试你的 dataloader 

你需要改写下述代码来测试 batching 函数，改写后的代码会现在的比较类似。

下面，我们生成了一些测试文本数据，并使用了一个你上面写 dataloader 。然后，我们会得到一些使用`sample_x`输入以及`sample_y`目标生成的文本。

你的代码会返回如下结果(通常是不同的顺序，如果你 shuffle 了你的数据):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### 大小
你的 sample_x 应该是 `(batch_size, sequence_length)`的 大小 或者是(10, 5)， sample_y 应该是 一维的: batch_size (10)。

### 值

你应该也会发现 sample_y, 是 test_text 数据中的*下一个*值。因此，对于一个输入的序列 `[ 28,  29,  30,  31,  32]` ，它的结尾是 `32`, 那么其相应的输出应该是 `33`。

In [13]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 12,  13,  14,  15,  16],
        [ 35,  36,  37,  38,  39],
        [ 43,  44,  45,  46,  47],
        [ 42,  43,  44,  45,  46],
        [ 27,  28,  29,  30,  31],
        [ 13,  14,  15,  16,  17],
        [ 33,  34,  35,  36,  37],
        [  7,   8,   9,  10,  11],
        [  0,   1,   2,   3,   4]])

torch.Size([10])
tensor([ 33,  17,  40,  48,  47,  32,  18,  38,  12,   5])


---
## 构建神经网络
使用 PyTorch [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module) 来实现一个 循环神经网络 RNN。你需要选择一个 GRU 或者 一个 LSTM。为了完成循环神经网络。为了实现 RNN，你需要实现以下类:
 - `__init__` - 初始化函数
 - `init_hidden` - LSTM/GRU 隐藏组昂泰的初始化函数
 - `forward` - 前向传播函数
 
初始化函数需要创建神经网络的层数，并保存到类。前向传播函数会使用这些网络来进行前向传播，并生成输出和隐藏状态。

在该流程完成后，**该模型的输出是 *最后的* 文字分数结果** 对于每段输入的文字序列，我们只需要输出一个单词，也就是，下一个单词。 

### 提示

1. 确保 lstm 的输出会链接一个 全链接层，你可以参考如下代码 `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. 你可以通过 reshape 模型最后输出的全链接层，来得到最终的文字分数:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [13]:
import torch.nn as nn
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.dropout_rate = dropout
        
        # define model layers
        
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, self.n_layers, dropout = self.dropout_rate, batch_first = True)
        self.fc = nn.Linear(self.hidden_dim, self.output_size)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function  
        
        batch_size = nn_input.size(0)
        
        embeds = self.embedding(nn_input)
        lstm_output, hidden = self.lstm(embeds, hidden)
        
        lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)
        
        output = self.fc(lstm_output)
        
        # reshape the output with schema of (batch_size, seq_length, output_size)
        output = output.view(batch_size, -1, self.output_size)
        
        # only last output matters 
        output = output[:, -1]
        
        # return one batch of output word scores and the hidden state
        return output, hidden
    
    
    def init_hidden(self, batch_size, use_gpu = True):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        
        weights = next(self.parameters()).data
        
        if use_gpu:
            hidden = (weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weights.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        
        else:
            hidden = (weights.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weights.new(self.n_layers, batch_size, self.hidden_dim).zero_())
            
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### 定义前向及后向传播

通过你实现的 RNN 类来进行前向及后项传播。你可以在训练循环中，不断地调用如下代码来实现：
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

函数中需要返回一个批次以及其隐藏状态的loss均值，你可以调用一个函数`RNN(inp, hidden)`来实现。记得，你可以通过调用`loss.item()` 来计算得到该loss。

**如果使用 GPU，你需要将你的数据存到 GPU 的设备上。**

In [14]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden, use_gpu = False):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    
    if use_gpu:
        rnn.cuda()
        inputs, target = inp.cuda(), target.cuda()
    else:
        rnn.cpu()
        inputs, target = inp.cpu(), target.cpu()
        
    # perform backpropagation and optimization        
    hidden = tuple([h.data for h in hidden])
    
    rnn.zero_grad()
    
    output, hidden = rnn(inputs, hidden)
    
    loss = criterion(output, target)
    
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 10)
    
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## 神经网络训练

神经网络结构完成以及数据准备完后，我们可以开始训练网络了。

### 训练循环

训练循环是通过 `train_decoder` 函数实现的。该函数将进行 epochs 次数的训练。模型的训练成果会在一定批次的训练后，被打印出来。这个“一定批次”可以通过`show_every_n_batches` 来设置。你会在下一节设置这个参数。

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100, use_gpu = False):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size, use_gpu)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden, use_gpu)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

In [25]:
def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100, use_gpu = False):
    batch_losses = []
    
    rnn.train()
    
    last_loss = np.inf

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size, use_gpu)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden, use_gpu)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
                
        now_loss = np.average(batch_losses)
        if now_loss < last_loss:
            helper.save_model('./save/trained_rnn', rnn)
            print("Saved model, loss {} -> {}".format(np.round(last_loss,4), np.round(now_loss,4)))
        last_loss = now_loss
            

    # returns a trained rnn
    return rnn

### 超参数

设置并训练以下超参数:
-  `sequence_length`，序列长度 
-  `batch_size`，分批大小
-  `num_epochs`，循环次数
-  `learning_rate`，Adam优化器的学习率
-  `vocab_size`，唯一标示词汇的数量
-  `output_size`，模型输出的大小 
-  `embedding_dim`，词嵌入的维度，小于 vocab_size
-  `hidden_dim`， 隐藏层维度
-  `n_layers`， RNN的层数
-  `show_every_n_batches`，打印结果的频次

如果模型没有获得你预期的结果，调整 `RNN`类中的上述参数。

In [16]:
# Data params
# Sequence Length
sequence_length = 10  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [17]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = len(vocab_to_int)
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### 训练
下一节，通过预处理数据来训练神经网络。如果你的loss结果不好，可以通过调整超参数来修正。通常情况下，大的隐藏层及层数会带来比较好的效果，但同时也会消耗较长的时间来训练。
> **你应该努力得到一个低于3.5的loss** 

你也可以试试不同的序列长度，该参数表明模型学习的范围大小。

In [40]:
use_gpu = torch.cuda.is_available()

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
rnn = helper.load_model('./save/trained_rnn')

if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches, use_gpu)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.459716830253601
Epoch:    1/10    Loss: 4.786456940174102
Epoch:    1/10    Loss: 4.60106560754776
Epoch:    1/10    Loss: 4.445810087680817
Epoch:    1/10    Loss: 4.3852531304359434
Epoch:    1/10    Loss: 4.3158373308181766
Epoch:    1/10    Loss: 4.295198258876801
Epoch:    1/10    Loss: 4.223955496311188
Epoch:    1/10    Loss: 4.212072878837586
Epoch:    1/10    Loss: 4.186242056846619
Epoch:    1/10    Loss: 4.17257838010788
Epoch:    1/10    Loss: 4.13766801404953
Epoch:    1/10    Loss: 4.11266220664978


/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Saved model, loss inf -> 4.1073
Epoch:    2/10    Loss: 4.017518603026683
Epoch:    2/10    Loss: 3.9247895669937134
Epoch:    2/10    Loss: 3.928374186515808
Epoch:    2/10    Loss: 3.9005954012870787
Epoch:    2/10    Loss: 3.9044028644561766
Epoch:    2/10    Loss: 3.8962726011276243
Epoch:    2/10    Loss: 3.8850488510131838
Epoch:    2/10    Loss: 3.8851635856628417
Epoch:    2/10    Loss: 3.8803118600845337
Epoch:    2/10    Loss: 3.8953558259010315
Epoch:    2/10    Loss: 3.8531425876617433
Epoch:    2/10    Loss: 3.873139783382416
Epoch:    2/10    Loss: 3.8848221192359924
Saved model, loss 4.1073 -> 3.849
Epoch:    3/10    Loss: 3.7653103556303296
Epoch:    3/10    Loss: 3.703661369800568
Epoch:    3/10    Loss: 3.706368790626526
Epoch:    3/10    Loss: 3.697054267883301
Epoch:    3/10    Loss: 3.6898650951385497
Epoch:    3/10    Loss: 3.710206111431122
Epoch:    3/10    Loss: 3.71174102306366
Epoch:    3/10    Loss: 3.728316292285919
Epoch:    3/10    Loss: 3.707179975032806

### 问题: 你如何决定你的模型超参数？
比如，你是否试过不同的 different sequence_lengths 并发现哪个使得模型的收敛速度变化？那你的隐藏层数和层数呢？你是如何决定使用这个网络参数的？

**答案:** (在这里写下)<br>
根据课程的建议，我了解到一般的embedding维度在200-300左右，所以我在调节embedding大小的时候就试了200和300两个数值，在收敛速度上面没有感觉到显著的变化。<br>
我试过三挡学习率，0.1,0.01,0.001。根据课程的建议这也是比较常用的选项。在实验中，我发现0.1的loss极不稳定，还有略微升高的倾斜，我判断是学习率太高可能导致了overshoot；0.1的学习率可以看到loss的一些下降，但是总体趋势并没有不断下降；所以我进一步调低学习率到0.001，看到了比较可观的下降，总体loss也会慢慢逐步下降。继而决定用0.001作为学习率。<br>
lstm的layer数量我有参考Andrej.K et al (2015)的实验，layer数一般为1,2,3。我测试了2和3的数值，感觉对下降没有特别大的差别，继而选择2作为layer数量，因为这样可以省略1/3的参数，提高网络的效率。<br>
在测试中，我认为 hidden 维度是对模型拟合比较重要的超参数，同样根据Andrej.K et al (2015)的实验结果，hidden在256同时layer为2时得到了较好的结果。<br>
最后 sequence lenght的影响我认为非常直观，因为length直接影响了lstm cell在预测时压缩的context的多少，过少容易欠拟合，过多容易过拟合。在试过10,20,30的数值之后，我认为20是一个比较好的数值。<br>
<br>
1. Andrej.K et al, 2015, Visualizing and understanding recurrent networks, LCLP 2016.


---
# 检查点

通过运行上面的训练单元，你的模型已经以`trained_rnn`名字存储，如果你存储了你的notebook， **你可以在之后的任何时间来访问你的代码和结果**. 下述代码可以帮助你重载你的结果!

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## 生成电视剧剧本
你现在可以生成你的“假”电视剧剧本啦！

### 生成文字
你的神经网络会不断重复生成一个单词，直到生成满足你要求长度的剧本。使用 `generate` 函数来完成上述操作。首先，使用 `prime_id` 来生成word id，之后确定生成文本长度 `predict_len`。同时， topk 采样来引入文字选择的随机性!

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### 生成一个新剧本
是时候生成一个剧本啦。设置`gen_length` 剧本长度，设置 `prime_word`为以下任意词来开始生成吧:
- "jerry"
- "elaine"
- "george"
- "kramer"

你可以把prime word 设置成 _任意 _ 单词, 但是使用名字开始会比较好(任何其他名字也是可以哒!)

In [21]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: $1.

jerry:(to kramer) hey, hey.(to jerry) you know...

jerry:(to jerry) you know, i have to go out with her.

helen: i know!(elaine enters)

elaine:(laughs) oh, well.

jerry: you know i just don't want to go out. you know, the only reason that i have to be able to get a little more wine in the city. you know, you know, you gotta have a lot of money.

elaine: i know, i know.

george: what?

jerry: what scare?

jerry:(to the phone) oh, hey.

jerry:(to george) you know, maybe i'll get a little more.

kramer: yeah, yeah. i don't know.

kramer: oh, yeah, well i got it.

jerry:(to george) i can't believe this is happening!

elaine:(laughs) what is that?

jerry: i dont know.

george:(to jerry) well, i think i'm not getting a job.

george: i know..... i got the job.

george: i know, i know..

george:(to george) oh, hi.

elaine: hi.

elaine:(to jerry) hey, what happened?

jerry: i dont know. i can't believe this. i can't hear anything. i was just a little bit.

jerry: you don't know, yo

In [30]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'morty' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


morty: squirrels.

jerry:(pointing) i don't even know how you can do the opposite.

elaine: yeah.

jerry: you think i have to be the ultimate section.

jerry: i mean, it's a miracle of the risotto.

jerry:(to george) hey, how ya doin'?

jerry:(to elaine) oh, my god, i gotta go.

elaine:(still looking at the counter) i don't think so.

kramer: yeah, yeah, well, i guess you could.

elaine: i don't know, i can't go to the bathroom. i mean, i think we can go to the movies tomorrow evening.

jerry: yeah!

elaine: what do you mean?

jerry: what?

jerry: i don't know, i can't. i don't think i should have the same time to get a ride.

jerry: well, i know.(to george) so i was gonna have my keys.(he walks away and heads around the table.) hey, what are you doing here?

jerry:(to george) you know what?

jerry:(to jerry) you know what? i don't want to talk, but, i'm sure, i'm just a little concerned, but, you know, if i had the idea that i was gonna go to a flea market.

kramer:(to george) hey.

j

#### 存下你最爱的片段

一旦你发现一段有趣或者好玩的片段，就把它存下啦！

In [ ]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# 这个电视剧剧本是无意义的
如果你的电视剧剧本不是很有逻辑也是ok的。下面是一个例子。

### 生成剧本案例

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.


如果这个电视剧剧本毫无意义，那也没有关系。我们的训练文本不到一兆字节。为了获得更好的结果，你需要使用更小的词汇范围或是更多数据。幸运的是，我们的确拥有更多数据！在本项目开始之初我们也曾提过，这是[另一个数据集](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data)的子集。我们并没有让你基于所有数据进行训练，因为这将耗费大量时间。然而，你可以随意使用这些数据训练你的神经网络。当然，是在完成本项目之后。
# 提交项目
在提交项目时，请确保你在保存 notebook 前运行了所有的单元格代码。请将 notebook 文件保存为 "dlnd_tv_script_generation.ipynb"，并将它作为 HTML 文件保存在 "File" -> "Download as" 中。请将 "helper.py" 和 "problem_unittests.py" 文件一并打包成 zip 文件提交。